In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import tensorflow as tf
import math
import tensorflow_addons as tfa
import random
import re
import csv
import itertools


from sklearn import metrics
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from mlxtend.evaluate import mcnemar_table
from statsmodels.stats.contingency_tables import mcnemar




from utils import (
    F1Score,
    plot_metrics,
    plot_accuracy,
    study_oriented_transformation,
    write_csv,
    prediction_results,
    plot_confusion_matrix,
    plot_contigency_table,
)


In [2]:
# To Activate GPU if there is
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


2022-10-17 03:14:02.633807: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-17 03:14:02.646597: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-17 03:14:02.647571: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
SEED = 1037

os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)
np.random.seed(SEED)


In [4]:
base_model = 'efficientnetv2-b3'

In [5]:
METRICS = [ 
    tf.keras.metrics.BinaryAccuracy(),
    tf.keras.metrics.Precision(name="precision"),
    tf.keras.metrics.Recall(name="recall"),
    tfa.metrics.CohenKappa(name="cohen_kappa", num_classes=2),
    F1Score(name="f1_score"),
]

STUDY_TYPES = [
    'elbow',
    'finger',
    'forearm',
    'hand',
    'humerus',
    'shoulder',
    'wrist',
]

CLASSES = ['NORMAL', 'ABNORMAL']


2022-10-17 03:14:02.862054: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-17 03:14:02.865438: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-17 03:14:02.866244: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-17 03:14:02.867035: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built witho

In [6]:
data_dir = 'dataset/local_radiographs/'
dirs = next(os.walk(data_dir))[1]
files = []

for sub in dirs:
    sub_dir = next(os.walk(os.path.join(data_dir, sub)))[1]
    root = next(os.walk(os.path.join(data_dir, sub)))[0]
    for item in sub_dir:
        rel_path = os.path.join(root, item)
        contents = next(os.walk(rel_path))[2]
        files.extend([os.path.join(rel_path, file) for file in contents])

local_test_img = pd.DataFrame(files, columns=['path'])        
local_test_img.head()

,path
0,dataset/local_radiographs/wrist/study15_positi...
1,dataset/local_radiographs/wrist/study15_positi...
2,dataset/local_radiographs/wrist/study16_negati...
3,dataset/local_radiographs/wrist/study16_negati...
4,dataset/local_radiographs/hand/study09_positiv...


In [15]:
local_test_img['label'] = local_test_img['path'].map(
    lambda x: '1' if 'positive' in x else '0'
)

local_test_img['study_type'] = local_test_img['path'].map(
    lambda x: x.split('/')[2]
)

local_test_img['study_path'] = local_test_img['path'].map(
    lambda x: x
)



local_test_img.head()


,path,label,study_type,study_path
0,dataset/local_radiographs/wrist/study15_positi...,1,wrist,dataset/local_radiographs/wrist/study15_positi...
1,dataset/local_radiographs/wrist/study15_positi...,1,wrist,dataset/local_radiographs/wrist/study15_positi...
2,dataset/local_radiographs/wrist/study16_negati...,0,wrist,dataset/local_radiographs/wrist/study16_negati...
3,dataset/local_radiographs/wrist/study16_negati...,0,wrist,dataset/local_radiographs/wrist/study16_negati...
4,dataset/local_radiographs/hand/study09_positiv...,1,hand,dataset/local_radiographs/hand/study09_positiv...


In [16]:
img_height = img_width = 300

def resize_img(img):
    try:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    except:
        print('error in resizing')
        img1 = isinstance(img, type(None))
        print('Does image is none: ', img1)
        print(img.shape)
    return cv2.resize(img, (img_height, img_width))

def canny_cropping(img):
    convert_img = np.array(img, dtype=np.uint8)

    gray = cv2.cvtColor(convert_img, cv2.COLOR_RGB2GRAY)


    ave_brightness = math.floor(np.average(gray))
    min_pixel = min(gray.flatten())

    edges = cv2.Canny(gray, min_pixel, ave_brightness)
    cnts = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

    for c in cnts:
        x, y, w, h = cv2.boundingRect(edges)
        gray = gray[y:y+h, x:x+w]
        break

    return gray

def apply_clahe(img):
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
    return clahe.apply(img.astype(np.uint8))

def preprocessing_without_clahe(img):
    cropped = canny_cropping(img)
    return resize_img(cropped)

def preprocessing_with_clahe(img):
    cropped = canny_cropping(img)
    clahe = apply_clahe(cropped)
    return resize_img(clahe)

In [17]:
batch = 8

test_batches_without_clahe = ImageDataGenerator(
    preprocessing_function=preprocessing_without_clahe
).flow_from_dataframe(
    target_size=(img_height, img_width),
    dataframe=local_test_img, 
    class_mode='binary',
    x_col='path',
    y_col='label',
    batch_size=batch, 
    shuffle=False)

Found 40 validated image filenames belonging to 2 classes.


In [10]:
model_without_clahe = tf.keras.models.load_model(
    'models/without_clahe/efficientnetv2-b3_sgd_std_lr_finetuned_v2.h5',
    custom_objects={'F1Score': F1Score}
)

In [19]:
predictions_without_clahe = model_without_clahe.predict(test_batches_without_clahe, verbose=1)

5/5 [==============================] - 2s 339ms/step


In [20]:
conv_prediction_without_clahe = local_test_img.copy()

conv_prediction_without_clahe['label'] = conv_prediction_without_clahe['label'].map(int)
conv_prediction_without_clahe['prediction'] = predictions_without_clahe.ravel()

In [21]:
conv_prediction_without_clahe = pd.DataFrame(
    [*study_oriented_transformation(conv_prediction_without_clahe)],
    columns=['study_type', 'study', 'label', 'prediction'],
)

In [22]:
results_without_clahe = prediction_results(conv_prediction_without_clahe)

write_csv(
    results_without_clahe, 
    f'testing_results/without_clahe/local_sample_general_prediction_results_{base_model}.csv'
)

print('=' * 52)
print(f'Prediction for all local radiographs {base_model} without CLAHE')
print('-----------------')
for result in results_without_clahe[:-1]:
    print(f"{result['metric'] + ' ' * (30 - len(result['metric']))}: {result['value']}")
print('=' * 52)


Prediction for all local radiographs efficientnetv2-b3 without CLAHE
-----------------
Cohen's kappa Coefficient (κ) : 0.23076923076923084
F1 Score                      : 0.6
Accuracy                      : 0.6
Precision                     : 0.75
Recall                        : 0.5


In [23]:
cm_general_without_clahe = results_without_clahe[-1]['value']
print(cm_general_without_clahe)
plot_confusion_matrix(
    cm_general_without_clahe, 
    CLASSES, 
    f'Local Radiographs Confusion Matrix for Model Without CLAHE {base_model}',
    False,
)

[[12  4]
 [12 12]]


<Figure size 432x288 with 0 Axes>

In [24]:
for body_part in STUDY_TYPES:
    parts = conv_prediction_without_clahe[conv_prediction_without_clahe['study_type'] == body_part]
    results = prediction_results(parts)
    write_csv(
        results_without_clahe, 
        f'testing_results/without_clahe/{body_part}__local_radiographs_prediction_results_{base_model}.csv'
    )
    parts_cm = results[-1]['value']
    plot_confusion_matrix(
        parts_cm, 
        CLASSES, 
        f'{body_part} Confusion Matrix for {base_model} Without CLAHE', 
        False
    )
    print('=' * 52)
    print(f'Prediction for {body_part} using {base_model} Without CLAHE')
    print('-----------------')
    for result in results[:-1]:
        print(f"{result['metric'] + ' ' * (30 - len(result['metric']))}: {result['value']}")
    print('=' * 52)


Prediction for elbow using efficientnetv2-b3 Without CLAHE
-----------------
Cohen's kappa Coefficient (κ) : 0.5
F1 Score                      : 0.6666666666666666
Accuracy                      : 0.75
Precision                     : 1.0
Recall                        : 0.5
Prediction for finger using efficientnetv2-b3 Without CLAHE
-----------------
Cohen's kappa Coefficient (κ) : 0.11764705882352933
F1 Score                      : 0.2857142857142857
Accuracy                      : 0.4444444444444444
Precision                     : 1.0
Recall                        : 0.16666666666666666
Prediction for forearm using efficientnetv2-b3 Without CLAHE
-----------------
Cohen's kappa Coefficient (κ) : 0.5
F1 Score                      : 0.8
Accuracy                      : 0.75
Precision                     : 0.6666666666666666
Recall                        : 1.0
Prediction for hand using efficientnetv2-b3 Without CLAHE
-----------------
Cohen's kappa Coefficient (κ) : 0.19354838709677413
F1 S

<Figure size 432x288 with 0 Axes>

In [25]:
batch = 8

test_batches_with_clahe = ImageDataGenerator(
    preprocessing_function=preprocessing_with_clahe
).flow_from_dataframe(
    target_size=(img_height, img_width),
    dataframe=local_test_img, 
    class_mode='binary',
    x_col='path',
    y_col='label',
    batch_size=batch, 
    shuffle=False)

Found 40 validated image filenames belonging to 2 classes.


In [26]:
model_with_clahe = tf.keras.models.load_model(
    'models/with_clahe/efficientnetv2-b3_sgd_std_lr_finetuned_v2.h5',
    custom_objects={'F1Score': F1Score}
)


In [27]:
predictions_with_clahe = model_with_clahe.predict(test_batches_with_clahe, verbose=1)

5/5 [==============================] - 4s 368ms/step


In [28]:
conv_prediction_with_clahe = local_test_img.copy()

conv_prediction_with_clahe['label'] = conv_prediction_with_clahe['label'].map(int)
conv_prediction_with_clahe['prediction'] = predictions_with_clahe.ravel()

In [29]:
conv_prediction_with_clahe = pd.DataFrame(
    [*study_oriented_transformation(conv_prediction_with_clahe)],
    columns=['study_type', 'study', 'label', 'prediction'],
)

In [30]:
results_with_clahe = prediction_results(conv_prediction_with_clahe)

write_csv(
    results_with_clahe, 
    f'testing_results/with_clahe/local_sample_general_prediction_results_{base_model}.csv'
)

print('=' * 52)
print(f'Prediction for all local radiographs {base_model} with CLAHE')
print('-' * 52)
for result in results_with_clahe[:-1]:
    print(f"{result['metric'] + ' ' * (30 - len(result['metric']))}: {result['value']}")
print('=' * 52)


Prediction for all local radiographs efficientnetv2-b3 with CLAHE
----------------------------------------------------
Cohen's kappa Coefficient (κ) : 0.3137254901960784
F1 Score                      : 0.6666666666666666
Accuracy                      : 0.65
Precision                     : 0.7777777777777778
Recall                        : 0.5833333333333334


In [31]:
cm_general_with_clahe = results_with_clahe[-1]['value']
print(cm_general_with_clahe)
plot_confusion_matrix(
    cm_general_with_clahe, 
    CLASSES, 
    f'Local Radiographs Confusion Matrix for {base_model} With CLAHE',
    True,
    )

[[12  4]
 [10 14]]


<Figure size 432x288 with 0 Axes>

In [32]:
for body_part in STUDY_TYPES:
    parts = conv_prediction_with_clahe[conv_prediction_with_clahe['study_type'] == body_part]
    results = prediction_results(parts)
    write_csv(
        results_with_clahe, 
        f'testing_results/with_clahe/{body_part}_local_radiographs_prediction_results_{base_model}.csv'
    )
    parts_cm = results[-1]['value']
    plot_confusion_matrix(
        parts_cm, 
        CLASSES, 
        f'{body_part} Confusion Matrix for {base_model} With CLAHE', 
        False
    )
    print('=' * 52)
    print(f'Prediction for {body_part} using {base_model} With CLAHE')
    print('-----------------')
    for result in results[:-1]:
        print(f"{result['metric'] + ' ' * (30 - len(result['metric']))}: {result['value']}")
    print('=' * 52)


Prediction for elbow using efficientnetv2-b3 With CLAHE
-----------------
Cohen's kappa Coefficient (κ) : 0.5
F1 Score                      : 0.6666666666666666
Accuracy                      : 0.75
Precision                     : 1.0
Recall                        : 0.5
Prediction for finger using efficientnetv2-b3 With CLAHE
-----------------
Cohen's kappa Coefficient (κ) : 0.0
F1 Score                      : 0.4444444444444444
Accuracy                      : 0.4444444444444444
Precision                     : 0.6666666666666666
Recall                        : 0.3333333333333333
Prediction for forearm using efficientnetv2-b3 With CLAHE
-----------------
Cohen's kappa Coefficient (κ) : 0.5
F1 Score                      : 0.8
Accuracy                      : 0.75
Precision                     : 0.6666666666666666
Recall                        : 1.0
Prediction for hand using efficientnetv2-b3 With CLAHE
-----------------
Cohen's kappa Coefficient (κ) : 0.2857142857142857
F1 Score           

<Figure size 432x288 with 0 Axes>